In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
cols="""duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())

columns.append('target')
#print(columns)
print(len(columns))

42


In [3]:
path = "./dataset/kdd/kddcup.data_10_percent.gz"
df = pd.read_csv(path,names=columns)

#Adding Attack Type column
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])


df

FileNotFoundError: [Errno 2] No such file or directory: './dataset/kdd/kddcup.data_10_percent.gz'

In [19]:
from sklearn.preprocessing import StandardScaler

no_normalize_cols = ['protocol_type','flag', 'service', 'land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login', 'target']
normalize_df = df.drop(no_normalize_cols, axis=1)

std_scaler = StandardScaler()

std_scaler.fit(normalize_df)

output = std_scaler.transform(normalize_df)

normalize_df = pd.DataFrame(output, columns=normalize_df.columns, index=list(normalize_df.index.values))

In [20]:
cols_protocol_type = pd.get_dummies(df.protocol_type, prefix='protocol_type')

In [21]:
cols_flag = pd.get_dummies(df.flag, prefix='flag')

In [22]:
cols_service = pd.get_dummies(df.service, prefix='service')

In [26]:
full_df = pd.concat([normalize_df, cols_protocol_type, cols_flag, cols_service, df['land'], df['logged_in'], df['root_shell'], df['su_attempted'], df['is_host_login'],df['is_guest_login'], df['target']], axis=1)

In [27]:
full_df

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,num_root,num_file_creations,...,service_uucp_path,service_vmnet,service_whois,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,target
0,-0.067792,-0.002879,0.138664,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
1,-0.067792,-0.002820,-0.011578,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
2,-0.067792,-0.002824,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
3,-0.067792,-0.002840,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
4,-0.067792,-0.002842,0.035214,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,-0.067792,-0.002748,0.030644,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
494017,-0.067792,-0.002776,0.042902,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
494018,-0.067792,-0.002856,0.010032,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.
494019,-0.067792,-0.002767,0.010032,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.00564,-0.011232,...,0,0,0,0,1,0,0,0,0,normal.


In [36]:
full_df['target'].value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: target, dtype: int64